In [142]:
from sympy import *
import time
import copy
import numpy as np

t = symbols('t')
j_m = symbols('j_m')  # j_m 是常数
a_m = symbols('a_m')  # a_m 是常数
v_m1 = symbols('v_m1')  # b_m 是常数
d = symbols('d')  # d 是常数

In [128]:
a0 = symbols('a_0')
a_seg1 = a0 + integrate(j_m, (t, 0, t))
a_seg1

a_0 + j_m*t

In [129]:
v0 = symbols('v_0')
v_seg1 = integrate(a_seg1, (t, 0, t)) + v0
v_seg1

a_0*t + j_m*t**2/2 + v_0

In [130]:
# [假设冲到 max]
int_seg1 = (a_m - a0) / j_m
int_seg2_suppose_a_m = a_m / j_m
a_seg2 = a_m - (t - int_seg1) * j_m
v_t2_suppose_a_m = integrate(a_seg1, (t, 0, int_seg1)) + \
    integrate(a_seg2, (t, int_seg1, int_seg1 + int_seg2_suppose_a_m))
v_t2_suppose_a_m = simplify(v_t2_suppose_a_m)
v_t2_suppose_a_m  # 假设直接冲到 a_m，最后 v 的速度

(-a_0**2/2 + a_m**2)/j_m

In [131]:
# v_t2_suppose_a_m <= v_m_1
# 那么需要冲到 v_m_1 然后保持 a_m，最后减速到 0
# int_seg1 = int_seg1
int_seg3 = int_seg2_suppose_a_m
int_seg2 = simplify((v_m1 - v_t2_suppose_a_m) / a_m)
int_seg2

(a_0**2/2 - a_m**2 + j_m*v_m1)/(a_m*j_m)

In [132]:
a_seg1 = a_seg1
a_seg2 = a_m
a_seg3 = a_m - (t - int_seg1 - int_seg2) * j_m
v_seg1 = v0 + integrate(a_seg1, (t, 0, t))
v_seg2 = simplify(v_seg1.subs(t, int_seg1) + integrate(a_seg2,
                                                       (t, int_seg1, t)))
v_seg3 = simplify(v_seg2.subs(t, int_seg1 + int_seg2) +
                  integrate(a_seg3, (t, int_seg1 + int_seg2, t)))
v_seg1, v_seg2, v_seg3 = simplify(
    v_seg1), simplify(v_seg2), simplify(v_seg3)
simplify(int_seg3 * j_m)

a_m

In [133]:
d_seg1 = integrate(v_seg1, (t, 0, t))
d_seg2 = simplify(d_seg1.subs(t, int_seg1) +
                  integrate(v_seg2, (t, int_seg1, t)))
d_seg3 = simplify(d_seg2.subs(t, int_seg1 + int_seg2) +
                  integrate(v_seg3, (t, int_seg1 + int_seg2, t)))
d_seg1, d_seg2, d_seg3 = simplify(d_seg1), simplify(d_seg2), simplify(d_seg3)
simplify(d_seg3.subs(t, int_seg1 + int_seg2 + int_seg3))
# 这只是第一种情况，注意嗷
# simplify(d_seg3.subs(t, int_seg1 + int_seg2 + int_seg3).subs(a0, 0).subs(v0, 0))

-a_0**4/(8*a_m*j_m**2) + a_0**3/(3*j_m**2) - a_0**2*a_m/(4*j_m**2) + a_0**2*v_0/(2*a_m*j_m) - a_0*v_0/j_m + a_m*v_0/j_m + a_m*v_m1/(2*j_m) + v_0*v_m1/a_m + v_m1**2/(2*a_m)

In [134]:
d_last = v_m1 / 2 * (v_m1 / a_m + a_m / j_m)
d_last

v_m1*(a_m/j_m + v_m1/a_m)/2

In [135]:
d_mid = simplify(d - d_last - d_seg3.subs(t, int_seg1 + int_seg2 + int_seg3))
d_mid

a_0**4/(8*a_m*j_m**2) - a_0**3/(3*j_m**2) + a_0**2*a_m/(4*j_m**2) - a_0**2*v_0/(2*a_m*j_m) + a_0*v_0/j_m - a_m*v_0/j_m - a_m*v_m1/j_m + d - v_0*v_m1/a_m - v_m1**2/a_m

In [136]:
equation = Eq(d_mid, 0)
solutions = solve(equation, v_m1)
simplify(solutions[0].subs(a0, 0).subs(v0, 0))

(-a_m**2 - sqrt(a_m*(a_m**3 + 4*d*j_m**2)))/(2*j_m)

In [137]:
solutions[1]

(-a_m**2/2 - j_m*v_0/2 + sqrt(18*a_0**4 - 48*a_0**3*a_m + 36*a_0**2*a_m**2 - 72*a_0**2*j_m*v_0 + 144*a_0*a_m*j_m*v_0 + 36*a_m**4 - 72*a_m**2*j_m*v_0 + 144*a_m*d*j_m**2 + 36*j_m**2*v_0**2)/12)/j_m

In [148]:
vs = {
    'd': 0.02,
    'j_m': 200,
    'a_m': 40,
    'v_m1': 1,
    'v0': -0,
    'a0': -0,
}


def subs(expr):
    for k, v in vs.items():
        expr = expr.subs(eval(k), v)
    return expr


def real(x):
    return float(subs(x).evalf())

In [149]:
t1 = sqrt((v_m1 / j_m))
t2 = 2 * t1
d1 = 1 / 6 * j_m * t1 ** 3
d2 = v_m1 * t1
a_in_t1 = j_m * t
a_in_t2 = a_in_t1.subs(t, t1) - j_m * (t - t1)
v_in_t1 = integrate(a_in_t1, (t, 0, t))
v_in_t2 = v_in_t1.subs(t, t1) + integrate(a_in_t2, (t, t1, t))
d_in_t1 = integrate(v_in_t1, (t, 0, t))
d_in_t2 = d_in_t1.subs(t, t1) + integrate(v_in_t2, (t, t1, t))

In [151]:
coe_expr = list(reversed(Poly(d_in_t2, t).all_coeffs()))
print(f'coe_expr: {coe_expr}')
# 以下为验证
coe = [real(c) for c in coe_expr]
d_tar = real(d_in_t2.subs(t, t1))
time_start = time.time()
coe[0] -= d_tar
coe = np.polynomial.Polynomial(coe)
polyder = coe.deriv()
root = opt.newton(lambda x: coe(x), x0=real(t1),
                  fprime=lambda x: polyder(x), tol=1e-5)
print(root, real(t1))
print('v:',  v_in_t2)
print('a: ', a_in_t2)

coe_expr: [j_m*(v_m1/j_m)**(3/2)/3, -v_m1, j_m*sqrt(v_m1/j_m), -j_m/6]
0.07071067811865475 0.07071067811865475
v: -j_m*t**2/2 + 2*j_m*t*sqrt(v_m1/j_m) - 2*j_m*v_m1/j_m + v_m1
a:  j_m*sqrt(v_m1/j_m) - j_m*(t - sqrt(v_m1/j_m))


In [14]:
print(d_in_t1.subs(t, t1))
print(simplify(d_in_t2.subs(t, t2)))

j_m*(v_m1/j_m)**(3/2)/6
v_m1*sqrt(v_m1/j_m)


In [15]:
# 根据当前 d 反推 t，得 v, a
eq = Eq(d_in_t1, d)
t1_sol = solve(eq, t)
print(t1_sol[0], v_in_t1)  # [p2]
d_in_t1.subs(t, t1_sol[0])

6**(1/3)*(d/j_m)**(1/3) j_m*t**2/2


d

In [91]:
v_in_t1, a_in_t1

(j_m*t**2/2, j_m*t)

In [17]:
eq = Eq(d_in_t2, d)
t2_sol = solve(eq, t)
subs(t1).evalf()
d_in_t2
Poly(d_in_t2, t).all_coeffs()

[-j_m/6, j_m*sqrt(v_m1/j_m), -v_m1, j_m*(v_m1/j_m)**(3/2)/3]

In [18]:
subs(t2_sol[0]).evalf()

0.0845932620387251 - 4.3e-19*I

In [19]:
subs(d2).evalf()

0.0707106781186548

In [20]:
import numpy as np
from scipy.optimize import fsolve

# 定义方程


def equation(x):
    return x**2 - 4


# 初始猜测值
x0 = 1

# 使用 fsolve 求解
solution = fsolve(equation, x0)
print("Solution:", solution)

Solution: [2.]


In [21]:
from scipy.optimize import root

coe = Poly(d_in_t2, t).all_coeffs()
coe.reverse()

coe = np.array([float(subs(c).evalf()) for c in coe])
d_tar = float(subs(d2).evalf())
coe[0] -= d_tar
coe = np.poly1d(coe)
polyder = np.polyder(coe)


t_start = time.time()
pts = np.roots(polyder)
print('time:', time.time() - t_start)
pts

time: 0.002777099609375


array([-19.31851653,   5.1763809 ])

In [ ]:
# [new]
int1 = a_m / j_m
int2 = v_m1 / a_m - a_m / j_m
int3 = a_m / j_m
d_in_t1.subs(t, int1)
a_in_t2 = a_m
v_in_t2 = v_in_t1.subs(t, int1) + integrate(a_in_t2, (t, int1, t))
a_in_t3 = a_m - j_m * (t - int2 - int1)
v_in_t3 = v_in_t2.subs(t, int1 + int2) + \
    integrate(a_in_t3, (t, int1 + int2, t))
d_in_t1 = integrate(v_in_t1, (t, 0, t))
d_in_t2 = d_in_t1.subs(t, int1) + integrate(v_in_t2, (t, int1, t))
d_in_t3 = d_in_t2.subs(t, int1 + int2) + \
    integrate(v_in_t3, (t, int1 + int2, t))
# d_in_t1, d_in_t2, d_in_t3
simplify(d_in_t2.subs(t, int1 + int2))

0.07071067811865475

In [23]:
# [解 t2 in int2]
eq = Eq(d_in_t3, d)
t_sol = solve(eq, t)
simplify(subs(t_sol[0].subs(d, d_in_t3.subs(t, int1 + int2 + int3))))
# print(t_sol[1])
# print(v_in_t2)
# print(a_in_t2)

3/20 - I**(1/3)/10 + I**(5/3)/10

In [24]:
v_in_t1.subs(t, int1), v_in_t2.subs(
    t, int1 + int2), a_in_t2.subs(t, int1 + int2)  # [p1]
subs(d_in_t1.subs(t, int1)).evalf(), subs(d_in_t2.subs(t, int1 + int2)
                                          ).evalf(), subs(d_in_t3.subs(t, int1 + int2 + int3)).evalf()

(0.00416666666666667, 0.0291666666666667, 0.0750000000000000)

In [25]:
print(d_in_t1.subs(t, int1))

a_m**3/(6*j_m**2)


In [86]:
real(d_in_t2.subs(t, int1 + int2))

0.029166666666666667

In [27]:
simplify(d_in_t3.subs(t, int1 + int2 + int3))

v_m1*(a_m**2 + j_m*v_m1)/(2*a_m*j_m)

In [155]:
coe_expr = list(reversed(Poly(d_in_t2, t).all_coeffs()))
print(f'coe_expr: {coe_expr}')
# 以下为验证
coe = [real(c) for c in coe_expr]
d_tar = real(d_in_t2.subs(t, int1 + int2))
time_start = time.time()
coe[0] -= d_tar
coe = np.polynomial.Polynomial(coe)
polyder = coe.deriv()
root = opt.newton(lambda x: coe(x), x0=real(int1),
                  fprime=lambda x: polyder(x), tol=1e-5)
print(root, real(int1 + int2))
print('v:',  v_in_t2)
print('a: ', a_in_t2)
print(int1 + int2 + int3)

coe_expr: [j_m*(v_m1/j_m)**(3/2)/3, -v_m1, j_m*sqrt(v_m1/j_m), -j_m/6]
0.024999999999909428 0.025
v: -j_m*t**2/2 + 2*j_m*t*sqrt(v_m1/j_m) - 2*j_m*v_m1/j_m + v_m1
a:  j_m*sqrt(v_m1/j_m) - j_m*(t - sqrt(v_m1/j_m))
a_m/j_m + v_m1/a_m


In [122]:
coe_expr = list(reversed(Poly(d_in_t3, t).all_coeffs()))
print(f'coe_expr: {coe_expr}')
# 以下为验证
coe = [real(c) for c in coe_expr]
d3_tar = real(d_in_t3.subs(t, int1 + int2 + int3))
coe[0] -= d3_tar
coe = np.polynomial.Polynomial(coe)
der = coe.deriv()
root = opt.newton(lambda x: coe(x), x0=real(int1 + int2),
                  fprime=lambda x: der(x), tol=1e-5)
print(coe)
print(root, real(int1 + int2 + int3))
print('v:', simplify(v_in_t3))
print('a:', a_in_t3)
simplify(v_in_t3)

coe_expr: [(a_m**6 + j_m**3*v_m1**3)/(6*a_m**3*j_m**2), (-a_m**4 - j_m**2*v_m1**2)/(2*a_m**2*j_m), (a_m**2 + j_m*v_m1)/(2*a_m), -j_m/6]
-0.0375 - 1.25·x + 15.0·x² - 33.33333333·x³
0.15000000000000002 0.15
v: -a_m**2/(2*j_m) + a_m*t - j_m*t**2/2 + j_m*t*v_m1/a_m - j_m*v_m1**2/(2*a_m**2)
a: a_m - j_m*(t - v_m1/a_m)


-a_m**2/(2*j_m) + a_m*t - j_m*t**2/2 + j_m*t*v_m1/a_m - j_m*v_m1**2/(2*a_m**2)